In [1]:
from datasets import load_dataset

# Load only hi-1.txt in streaming mode
streamed_dataset = load_dataset(
    "text",
    data_files="https://huggingface.co/datasets/ai4bharat/IndicCorpV2/resolve/main/data/hi-1.txt",
    streaming=True
)

# Get the training split (default for text datasets)
data = streamed_dataset['train']


c:\Users\dhruv\Downloads\NLP_SVNIT\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Iterate through the first few rows of the streamed dataset
print("First 10 rows of the streamed dataset:")
for i, row in enumerate(data):
    if i >= 10:
        break
    print(row)

First 10 rows of the streamed dataset:
{'text': 'लोगों को बिलों संबंधी सुविधा देना ही उनका काम'}
{'text': ''}
{'text': 'इनेलो 1987 में उस वक्त ऐसे ही दोराहे पर खड़ी थी, जब पूर्व उपप्रधानमंत्री देवीलाल ने अपने पुत्र ओमप्रकाश चौटाला को अपना राजनीतिक उत्तराधिकारी घोषित किया था। हालांकि तब पार्टी पर देवीलाल की मजबूत पकड़ के चलते पार्टी टूटने से बच गई थी। 1989 में देवीलाल केन्द्र की राजनीति में सक्रिय हो गए थे और उनके उपप्रधानमंत्री बनने के पश्चात् उनके तीन बेटों जगदीश सिंह, रणजीत सिंह और ओमप्रकाश चौटाला में से रणजीत और ओमप्रकाश के बीच हरियाणा में उनकी राजनीतिक विरासत को लेकर जंग शुरू हो गई थी। उन परिस्थितियों में देवीलाल ने कड़ा निर्णय लेते हुए पार्टी की बागडोर ओमप्रकाश चौटाला के हवाले कर दी थी, जिसके बाद रणजीत की बगावत का असर पार्टी, संगठन और उनकी सरकार पर भी पड़ा था। उस समय रणजीत की नाराजगी के चलते उनके समर्थन में कई कैबिनेट मंत्रियों ने इस्तीफे दे दिए थे किन्तु तब पार्टी सुप्रीमो चौ. देवीलाल की हरियाणा की जनता पर इतनी मजबूत पकड़ थी कि ओमप्रकाश चौटाला को उत्तराधिकारी बनाने के उनके फैसले 

In [3]:
import re

# Regex for special patterns
URL_PATTERN = r"https?://\S+|www\.\S+"
EMAIL_PATTERN = r"\b[\w\.-]+?@\w+?\.\w+?\b"
DECIMAL_PATTERN = r"\b\d+\.\d+\b"
DATE_PATTERN = r"\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b"

# Unicode ranges
DEVANAGARI_WORD_PATTERN = r"[\u0900-\u097F]+"
ENGLISH_WORD_PATTERN = r"[a-zA-Z0-9]+"
PUNCTUATION_PATTERN = r"[^\w\s\u0900-\u097F]"

# Sentence tokenizer: split at । ! ? followed by space
def sentence_tokenize(text):
    return re.split(r'(?<=[।!?])\s+', text.strip())

# Word tokenizer: special tokens + Devanagari words + punctuation
def word_tokenize(text):
    special_tokens = re.findall(f"{EMAIL_PATTERN}|{URL_PATTERN}|{DATE_PATTERN}|{DECIMAL_PATTERN}", text)

    # Remove special tokens to avoid duplication
    temp_text = text
    for token in special_tokens:
        temp_text = temp_text.replace(token, "")

    tokens = re.findall(
        f"{DEVANAGARI_WORD_PATTERN}|{ENGLISH_WORD_PATTERN}|{PUNCTUATION_PATTERN}",
        temp_text
    )
    return special_tokens + tokens


In [4]:
# Output file
output_file = "tokenized_hi_1_1000.txt"

# Stats counters
total_sentences = 0
total_words = 0
total_chars = 0
unique_tokens = set()

# Process first 1000 rows
with open(output_file, "w", encoding="utf-8") as f:
    for i, sample in enumerate(data):
        if i >= 1000:
            break

        paragraph = sample['text']
        if not paragraph.strip():
            continue

        # Sentence and word tokenization
        sentences = sentence_tokenize(paragraph)
        total_sentences += len(sentences)

        for sentence in sentences:
            tokens = word_tokenize(sentence)
            total_words += len(tokens)
            total_chars += sum(len(token) for token in tokens)
            unique_tokens.update(tokens)

            # Write sentence tokens to file
            f.write(" ".join(tokens) + "\n")
            # for word in tokens:
            #   f.write(word + "\n")


In [5]:
# Compute metrics
avg_sentence_length = total_words / total_sentences if total_sentences else 0
avg_word_length = total_chars / total_words if total_words else 0
ttr = len(unique_tokens) / total_words if total_words else 0

# Print stats
print("✅ Processed 1000 rows from hi-1.txt")
print("📊 Corpus Statistics:")
print(f"Total sentences: {total_sentences}")
print(f"Total words: {total_words}")
print(f"Total characters: {total_chars}")
print(f"Average sentence length: {avg_sentence_length:.2f} words")
print(f"Average word length: {avg_word_length:.2f} characters")
print(f"Type/Token Ratio (TTR): {ttr:.4f}")


✅ Processed 1000 rows from hi-1.txt
📊 Corpus Statistics:
Total sentences: 1466
Total words: 33152
Total characters: 126918
Average sentence length: 22.61 words
Average word length: 3.83 characters
Type/Token Ratio (TTR): 0.2104
